In [1]:
import os
if not os.path.exists("logs"):
    os.makedirs("logs")


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time
from sklearn.model_selection import train_test_split
import numpy as np
import os

# Загрузка данных из pickle-файлов
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

# Нормализация данных
X = X / 255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            # Создание модели
            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for _ in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            # Компиляция модели
            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'])

            # Разделение данных на обучающий и валидационный наборы
            X_train, X_val, y_train, y_val = train_test_split(np.array(X), np.array(y), test_size=0.3, random_state=42)

            # Обучение модели
            model.fit(X_train, y_train,
                      batch_size=32,
                      epochs=10,
                      validation_data=(X_val, y_val))#callbacks=[tensorboard]

            # Сохранение модели
            model.save('64x3-CNN.keras')


3-conv-64-nodes-0-dense-1711976830
Epoch 1/10
524/524 [==============================] - 31s 57ms/step - loss: 0.6457 - accuracy: 0.6161 - val_loss: 0.6028 - val_accuracy: 0.6764
Epoch 2/10
524/524 [==============================] - 30s 57ms/step - loss: 0.5614 - accuracy: 0.7115 - val_loss: 0.5356 - val_accuracy: 0.7335
Epoch 3/10
524/524 [==============================] - 29s 56ms/step - loss: 0.5084 - accuracy: 0.7533 - val_loss: 0.5266 - val_accuracy: 0.7374
Epoch 4/10
524/524 [==============================] - 31s 59ms/step - loss: 0.4720 - accuracy: 0.7748 - val_loss: 0.5172 - val_accuracy: 0.7426
Epoch 5/10
524/524 [==============================] - 32s 62ms/step - loss: 0.4437 - accuracy: 0.7918 - val_loss: 0.4722 - val_accuracy: 0.7698
Epoch 6/10
524/524 [==============================] - 31s 58ms/step - loss: 0.4164 - accuracy: 0.8089 - val_loss: 0.4598 - val_accuracy: 0.7833
Epoch 7/10
524/524 [==============================] - 30s 57ms/step - loss: 0.3897 - accuracy: 0.8219

In [4]:
import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]  # will use this to convert prediction num to string value


def prepare(filepath):
    IMG_SIZE = 50  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # return the image with shaping that TF wants.

In [5]:
model = tf.keras.models.load_model("64x3-CNN.keras")

In [7]:
prediction = model.predict([prepare('9589.jpg')])  # REMEMBER YOU'RE PASSING A LIST OF THINGS YOU WISH TO PREDICT

1/1 [==============================] - 0s 69ms/step


In [8]:
print(CATEGORIES[int(prediction[0][0])])

Cat


In [9]:
prediction = model.predict([prepare('11317.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

1/1 [==============================] - 0s 20ms/step
[[1.]]
Cat


In [10]:
import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]


def prepare(filepath):
    IMG_SIZE = 50  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


model = tf.keras.models.load_model("64x3-CNN.keras")

prediction = model.predict([prepare('11317.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

1/1 [==============================] - 0s 79ms/step
[[1.]]
Cat
